# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Using modelling and mapping for digital insights into diseases in the rice field	Invited talk	kspp_2016	2016 Korean Society of Plant Pathology Fall Meeting and International Conference	2016-10-21	Pyeongchang, Gangwon-do, Korea		
Mapping Rice Diseases for Targeted Deployment of Resistant Varieties in India	Talk	apps_2015	Australasian Plant Pathology Society Biennial Meeting	2015-09-15	Fremantle, Western Australia, Australia		
Modeling the impact of disease resistance on rice yields in the Philippines and Indonesia	Talk	apps_2013	Australasian Plant Pathology Society Biennial Meeting	2013-03-01	Auckland, New Zealand		
Spatial modelling of rice yield losses due to bacterial leaf blight and leaf blast in a changing climate	Talk	icpp_2013	10th International Congress of Plant Pathology	2014-03-01	Beijing, China		
Preventing what ails rice with a strategic, statistical, prescriptive model system	Talk	aps_2012	American Phytopathologic

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Using modelling and mapping for digital insigh...,Invited talk,kspp_2016,2016 Korean Society of Plant Pathology Fall Me...,2016-10-21,"Pyeongchang, Gangwon-do, Korea",NaN,NaN
1,Mapping Rice Diseases for Targeted Deployment ...,Talk,apps_2015,Australasian Plant Pathology Society Biennial ...,2015-09-15,"Fremantle, Western Australia, Australia",NaN,NaN
2,Modeling the impact of disease resistance on r...,Talk,apps_2013,Australasian Plant Pathology Society Biennial ...,2013-03-01,"Auckland, New Zealand",NaN,NaN
3,Spatial modelling of rice yield losses due to ...,Talk,icpp_2013,10th International Congress of Plant Pathology,2014-03-01,"Beijing, China",NaN,NaN
4,"Preventing what ails rice with a strategic, st...",Talk,aps_2012,American Phytopathological Society Annual Meeting,2012-08-05,"Providence, Rhode Island, USA",NaN,NaN
5,Adapting global disease forecasting models for...,Talk,iep_2009,10th International Epidemiology Workshop,2009-06-09,"Geneva, NY, USA",NaN,NaN
6,Taking sustainable crop protection from the f...,Invited talk,irc_2014,4th International Rice Congress (IRC2014),2014-10-24,"Bangkok, Thailand",https://figshare.com/articles/Taking_Sustainab...,NaN
7,Impact of climate change on rice diseases,Invited talk,cabi_2014,Workshop on the impact of climate change on cr...,2014-08-14,"Ho Chi Minh City, Vietnam",NaN,NaN
8,Epidemiology and disease management of rice br...,Invited talk,ips_2014,66th Annual Indian Phytopathological Society M...,2014-06-15,"Raipur, Chhattisgarh, India",https://figshare.com/articles/IPS_2014/1037569,NaN
9,Biosecurity risks in Southeast Asia impacting ...,Invited talk,pacom_2013,Australian Department of Defence (ADoD) and U....,2013-04-13,"Sydney, New South Wales, Australia",NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

2009-06-09-iep_2009.md      2014-10-24-irc_2014.md
2010-03-17-nyas_2010.md     2015-09-15-apps_2015.md
2012-08-05-aps_2012.md      2016-10-21-kspp_2016.md
2013-03-01-apps_2013.md     2017-07-19-GRDC_2017.md
2013-04-13-pacom_2013.md    2017-09-26-SciPlant_2017.md
2013-05-28-irri_2013.md     talkmap.ipynb
2014-03-01-icpp_2013.md     talkmap.py
2014-06-15-ips_2014.md      talks.ipynb
2014-08-14-cabi_2014.md


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
